# Gridsearch Cluster all small dag generation evaluation

In [1]:
%load_ext autoreload
%autoreload 2
from clustercausal.experiments.ExperimentRunner import ExperimentRunner
import itertools
import yaml
import os
import pickle
import pandas as pd
from clustercausal.utils.Utils import *
from clustercausal.experiments.Utils import load_experiment_folder
pd.set_option('display.max_columns', None)

Detecting 1 CUDA device(s).
2023-11-08 16:56:21,804 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\rpy2\situation.py[line:41] - INFO: cffi mode is CFFI_MODE.ANY
2023-11-08 16:56:21,804 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\rpy2\situation.py[line:218] - INFO: R home found: C:\Program Files\R\R-4.3.1
2023-11-08 16:56:21,833 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\rpy2\rinterface_lib\embedded.py[line:20] - INFO: Default options to initialize R: rpy2, --quiet, --no-save
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [3]:
# Can take a longer time depending on the numer of experiments, ca. 5sec for 100 experiments
data = load_experiment_folder('clustercausal\experiments\_results')

### Data exploration

In [4]:
data[:5]

,base_adj_f1_score,base_adj_false_negative,base_adj_false_positive,base_adj_precision,base_adj_recall,base_adj_true_negative,base_adj_true_positive,base_arrow_f1_score,base_arrow_f1_score_ce,base_arrow_false_negative,base_arrow_false_negative_ce,base_arrow_false_positive,base_arrow_false_positive_ce,base_arrow_precision,base_arrow_precision_ce,base_arrow_recall,base_arrow_recall_ce,base_arrow_true_negative,base_arrow_true_negative_ce,base_arrow_true_positive,base_arrow_true_positive_ce,base_base_shd,base_sid_lower,cluster_adj_f1_score,cluster_adj_false_negative,cluster_adj_false_positive,cluster_adj_precision,cluster_adj_recall,cluster_adj_true_negative,cluster_adj_true_positive,cluster_arrow_f1_score,cluster_arrow_f1_score_ce,cluster_arrow_false_negative,cluster_arrow_false_negative_ce,cluster_arrow_false_positive,cluster_arrow_false_positive_ce,cluster_arrow_precision,cluster_arrow_precision_ce,cluster_arrow_recall,cluster_arrow_recall_ce,cluster_arrow_true_negative,cluster_arrow_true_negative_ce,cluster_arrow_true_positive,cluster_arrow_true_positive_ce,cluster_cluster_shd,cluster_sid_lower,pruned_base_adj_f1_score,pruned_base_adj_false_negative,pruned_base_adj_false_positive,pruned_base_adj_precision,pruned_base_adj_recall,pruned_base_adj_true_negative,pruned_base_adj_true_positive,pruned_base_arrow_f1_score,pruned_base_arrow_f1_score_ce,pruned_base_arrow_false_negative,pruned_base_arrow_false_negative_ce,pruned_base_arrow_false_positive,pruned_base_arrow_false_positive_ce,pruned_base_arrow_precision,pruned_base_arrow_precision_ce,pruned_base_arrow_recall,pruned_base_arrow_recall_ce,pruned_base_arrow_true_negative,pruned_base_arrow_true_negative_ce,pruned_base_arrow_true_positive,pruned_base_arrow_true_positive_ce,pruned_base_pruned_base_shd,pruned_base_sid_lower,alpha,dag_method,distribution_type,edge_ratios,empty_graph_shd,indep_test,n_clusters,n_edges,n_nodes,noise_scale,sample_size,scm_method,seed,true_sid_lower,true_sid_upper,weight_range
0,1.000000,0.0,0.0,1.000000,1.000000,36.0,9.0,0.714286,0.714286,4.0,4.0,0.0,0.0,1.000000,1.0,0.555556,0.555556,91.0,91.0,5.0,5.0,0.0,NaN,1.000000,0.0,0.0,1.000000,1.000000,36.0,9.0,0.800000,0.800000,3.0,3.0,0.0,0.0,1.000000,1.0,0.666667,0.666667,91.0,91.0,6.0,6.0,0.0,NaN,1.000000,0.0,0.0,1.000000,1.000000,36.0,9.0,0.714286,0.714286,4.0,4.0,0.0,0.0,1.000000,1.0,0.555556,0.555556,91.0,91.0,5.0,5.0,0.0,NaN,0.01,erdos_renyi,exp,"[0.09, 0.25, 0.25]",9.0,fisherz,2.0,10.0,10.0,1.0,1000.0,linear,None,NaN,NaN,"(-1, 2)"
1,0.769231,2.0,1.0,0.833333,0.714286,37.0,5.0,0.444444,0.571429,5.0,3.0,0.0,0.0,1.000000,1.0,0.285714,0.400000,93.0,95.0,2.0,2.0,3.0,NaN,0.769231,2.0,1.0,0.833333,0.714286,37.0,5.0,0.615385,0.800000,3.0,1.0,2.0,1.0,0.666667,0.8,0.571429,0.800000,91.0,94.0,4.0,4.0,4.0,NaN,0.769231,2.0,1.0,0.833333,0.714286,37.0,5.0,0.444444,0.571429,5.0,3.0,0.0,0.0,1.000000,1.0,0.285714,0.400000,93.0,95.0,2.0,2.0,3.0,NaN,0.05,erdos_renyi,exp,"[0.07, 0.21, 0.21]",7.0,fisherz,2.0,10.0,10.0,1.0,1000.0,linear,None,NaN,NaN,"(-1, 2)"
2,0.823529,2.0,1.0,0.875000,0.777778,35.0,7.0,0.666667,0.833333,4.0,2.0,1.0,0.0,0.833333,1.0,0.555556,0.714286,90.0,93.0,5.0,5.0,3.0,NaN,0.823529,2.0,1.0,0.875000,0.777778,35.0,7.0,0.750000,0.923077,3.0,1.0,1.0,0.0,0.857143,1.0,0.666667,0.857143,90.0,93.0,6.0,6.0,3.0,NaN,0.823529,2.0,1.0,0.875000,0.777778,35.0,7.0,0.666667,0.833333,4.0,2.0,1.0,0.0,0.833333,1.0,0.555556,0.714286,90.0,93.0,5.0,5.0,3.0,NaN,0.10,erdos_renyi,exp,"[0.35, 0.22, 0.22]",9.0,fisherz,2.0,10.0,10.0,1.0,1000.0,linear,None,NaN,NaN,"(-1, 2)"
3,0.947368,1.0,0.0,1.000000,0.900000,35.0,9.0,0.461538,0.500000,7.0,6.0,0.0,0.0,1.000000,1.0,0.300000,0.333333,90.0,91.0,3.0,3.0,1.0,NaN,0.900000,1.0,1.0,0.900000,0.900000,34.0,9.0,0.888889,0.941176,2.0,1.0,0.0,0.0,1.000000,1.0,0.800000,0.888889,90.0,91.0,8.0,8.0,2.0,NaN,0.947368,1.0,0.0,1.000000,0.900000,35.0,9.0,0.461538,0.500000,7.0,6.0,0.0,0.0,1.000000,1.0,0.300000,0.333333,90.0,91.0,3.0,3.0,1.0,NaN,0.25,erdos_renyi,exp,"[0.2, 0.17, 0.17]",10.0,fisherz,2.0,10.0,

In [5]:
data.describe()

,base_adj_f1_score,base_adj_false_negative,base_adj_false_positive,base_adj_precision,base_adj_recall,base_adj_true_negative,base_adj_true_positive,base_arrow_f1_score,base_arrow_f1_score_ce,base_arrow_false_negative,base_arrow_false_negative_ce,base_arrow_false_positive,base_arrow_false_positive_ce,base_arrow_precision,base_arrow_precision_ce,base_arrow_recall,base_arrow_recall_ce,base_arrow_true_negative,base_arrow_true_negative_ce,base_arrow_true_positive,base_arrow_true_positive_ce,base_base_shd,base_sid_lower,cluster_adj_f1_score,cluster_adj_false_negative,cluster_adj_false_positive,cluster_adj_precision,cluster_adj_recall,cluster_adj_true_negative,cluster_adj_true_positive,cluster_arrow_f1_score,cluster_arrow_f1_score_ce,cluster_arrow_false_negative,cluster_arrow_false_negative_ce,cluster_arrow_false_positive,cluster_arrow_false_positive_ce,cluster_arrow_precision,cluster_arrow_precision_ce,cluster_arrow_recall,cluster_arrow_recall_ce,cluster_arrow_true_negative,cluster_arrow_true_negative_ce,cluster_arrow_true_positive,cluster_arrow_true_positive_ce,cluster_cluster_shd,cluster_sid_lower,pruned_base_adj_f1_score,pruned_base_adj_false_negative,pruned_base_adj_false_positive,pruned_base_adj_precision,pruned_base_adj_recall,pruned_base_adj_true_negative,pruned_base_adj_true_positive,pruned_base_arrow_f1_score,pruned_base_arrow_f1_score_ce,pruned_base_arrow_false_negative,pruned_base_arrow_false_negative_ce,pruned_base_arrow_false_positive,pruned_base_arrow_false_positive_ce,pruned_base_arrow_precision,pruned_base_arrow_precision_ce,pruned_base_arrow_recall,pruned_base_arrow_recall_ce,pruned_base_arrow_true_negative,pruned_base_arrow_true_negative_ce,pruned_base_arrow_true_positive,pruned_base_arrow_true_positive_ce,pruned_base_pruned_base_shd,pruned_base_sid_lower,alpha,empty_graph_shd,n_clusters,n_edges,n_nodes,noise_scale,sample_size,true_sid_lower,true_sid_upper
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,707.000000,707.000000,720.000000,720.000000,720.000000,720.000000,713.000000,713.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,0.0,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,0.0,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,707.000000,707.000000,720.000000,720.000000,720.000000,720.000000,713.000000,713.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,0.0,720.000000,720.000000,720.000000,720.000000,720.0,720.0,720.0,0.0,0.0
mean,0.662502,13.920833,1.241667,0.913248,0.555165,17.423611,12.413889,0.412875,0.635763,19.026389,5.105556,4.191667,3.101389,0.647664,0.709374,0.314406,0.574666,69.473611,84.484722,7.308333,7.308333,18.263889,NaN,0.738102,10.658333,1.927778,0.890791,0.661659,16.737500,15.676389,0.653066,0.880253,12.923611,2.265278,3.086111,1.330556,0.815988,0.912189,0.571407,0.855009,70.579167,82.993056,13.411111,13.411111,13.916667,NaN,0.662502,13.920833,1.241667,0.913248,0.555165,17.423611,12.413889,0.412875,0.635763,19.026389,5.105556,4.191667,3.101389,0.647664,0.709374,0.314406,0.574666,69.473611,84.484722,7.308333,7.308333,18.263889,NaN,0.182000,26.334722,3.500000,27.500000,10.0,1.0,1000.0,NaN,NaN
std,0.153845,9.794444,1.509336,0.098910,0.212982,11.310868,3.023474,0.165913,0.190751,10.369696,2.440382,2.983858,2.515487,0.220625,0.219030,0.165103,0.205744,12.344132,4.634309,3.396298,3.396298,10.387610,NaN,0.114856,7.947669,2.059695,0.110920,0.179960,11.179991,4.947232,0.156227,0.120173,8.937178,2.097860,2.548190,1.705480,0.144263,0.111657,0.196281,0.134665,11.995913,5.634062,4.795442,4.795442,8.426975,NaN,0.153845,9.794444,1.509336,0.098910,0.212982,11.310868,3.023474,0.165913,0.190751,10.369696,2.440382,2.983858,2.515487,0.220625,0.219030,0.165103,0.205744,12.344132,4.634309,3

In [6]:
prec_recall_metrics = ['base_adj_precision', 'pruned_base_adj_precision', 'cluster_adj_precision', \
           'base_adj_recall', 'pruned_base_adj_recall','cluster_adj_recall', \
           'base_adj_f1_score', 'pruned_base_adj_f1_score', 'cluster_adj_f1_score']
data[prec_recall_metrics].describe()

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,0.913248,0.913248,0.890791,0.555165,0.555165,0.661659,0.662502,0.662502,0.738102
std,0.098910,0.098910,0.110920,0.212982,0.212982,0.179960,0.153845,0.153845,0.114856
min,0.350000,0.350000,0.304348,0.177778,0.177778,0.288889,0.301887,0.301887,0.431373
25%,0.857143,0.857143,0.833333,0.384615,0.384615,0.520000,0.540021,0.540021,0.656716
50%,0.928571,0.928571,0.909091,0.486833,0.486833,0.638889,0.637331,0.637331,0.733333
75%,1.000000,1.000000,1.000000,0.705882,0.705882,0.777778,0.774194,0.774194,0.816790
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
count_metrics = ['base_adj_false_negative', 'pruned_base_adj_false_negative', 'cluster_adj_false_negative', \
                 'base_adj_false_positive', 'pruned_base_adj_false_positive','cluster_adj_false_positive', \
                    'base_adj_true_negative', 'pruned_base_adj_true_negative', 'cluster_adj_true_negative', \
                        'base_adj_true_positive', 'pruned_base_adj_true_positive', 'cluster_adj_true_positive']
data[count_metrics]

,base_adj_false_negative,pruned_base_adj_false_negative,cluster_adj_false_negative,base_adj_false_positive,pruned_base_adj_false_positive,cluster_adj_false_positive,base_adj_true_negative,pruned_base_adj_true_negative,cluster_adj_true_negative,base_adj_true_positive,pruned_base_adj_true_positive,cluster_adj_true_positive
0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,36.0,36.0,9.0,9.0,9.0
1,2.0,2.0,2.0,1.0,1.0,1.0,37.0,37.0,37.0,5.0,5.0,5.0
2,2.0,2.0,2.0,1.0,1.0,1.0,35.0,35.0,35.0,7.0,7.0,7.0
3,1.0,1.0,1.0,0.0,0.0,1.0,35.0,35.0,34.0,9.0,9.0,9.0
4,0.0,0.0,0.0,2.0,2.0,2.0,32.0,32.0,32.0,11.0,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
715,15.0,15.0,10.0,1.0,1.0,1.0,19.0,19.0,19.0,10.0,10.0,15.0
716,13.0,13.0,8.0,1.0,1.0,0.0,19.0,19.0,20.0,12.0,12.0,17.0
717,9.0,9.0,10.0,2.0,2.0,3.0,18.0,18.0,17.0,16.0,16.0,15.0
718,11.0,11.0,6.0,1.0,1.0,3.0,19.0,19.0,17.0,14.0,14.0,19.0


In [8]:
dist_metrics = ['base_base_shd', 'pruned_base_pruned_base_shd', 'cluster_cluster_shd']#, \
         #   'base_sid_lower', 'base_sid_upper', 'pruned_base_sid_lower', 'pruned_base_sid_upper', \
         #      'cluster_sid_lower', 'cluster_sid_upper']
data[dist_metrics].describe()

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
count,720.000000,720.000000,720.000000
mean,18.263889,18.263889,13.916667
std,10.387610,10.387610,8.426975
min,0.000000,0.000000,0.000000
25%,10.000000,10.000000,7.000000
50%,19.000000,19.000000,14.000000
75%,26.000000,26.000000,20.000000
max,41.000000,41.000000,39.000000


In [9]:
edge_ratios = data['edge_ratios'].apply(pd.Series)
edge_ratios.columns = ['intra_cluster', 'inter_cluster', 'inter_including_disconnected_cluster']
mean_values = edge_ratios.mean()
mean_values = pd.DataFrame([mean_values])
print('Edge Ratios \n', mean_values)

Edge Ratios 
    intra_cluster  inter_cluster  inter_including_disconnected_cluster
0       0.486292       0.659181                              0.605625


### Comparing true & false positives and true & false negatives

In [10]:
true_false_positive_metrics = ['base_adj_true_positive', 'pruned_base_adj_true_positive', 'cluster_adj_true_positive', \
                          'base_adj_false_positive', 'pruned_base_adj_false_positive','cluster_adj_false_positive']

true_false_negative_metrics = ['base_adj_true_negative', 'pruned_base_adj_true_negative', 'cluster_adj_true_negative', \
                          'base_adj_false_negative', 'pruned_base_adj_false_negative', 'cluster_adj_false_negative']

true_false_metrics = ['base_adj_true_positive', 'pruned_base_adj_true_positive', 'cluster_adj_true_positive', \
                    'base_adj_true_negative', 'pruned_base_adj_true_negative', 'cluster_adj_true_negative', \
                    'base_adj_false_positive', 'pruned_base_adj_false_positive','cluster_adj_false_positive', \
                    'base_adj_false_negative', 'pruned_base_adj_false_negative', 'cluster_adj_false_negative']

In [11]:
data[true_false_positive_metrics].describe()

,base_adj_true_positive,pruned_base_adj_true_positive,cluster_adj_true_positive,base_adj_false_positive,pruned_base_adj_false_positive,cluster_adj_false_positive
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,12.413889,12.413889,15.676389,1.241667,1.241667,1.927778
std,3.023474,3.023474,4.947232,1.509336,1.509336,2.059695
min,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000
25%,10.000000,10.000000,12.000000,0.000000,0.000000,0.000000
50%,12.000000,12.000000,16.000000,1.000000,1.000000,1.000000
75%,14.000000,14.000000,19.000000,2.000000,2.000000,3.000000
max,22.000000,22.000000,31.000000,13.000000,13.000000,16.000000


In [12]:
data[true_false_negative_metrics].describe()

,base_adj_true_negative,pruned_base_adj_true_negative,cluster_adj_true_negative,base_adj_false_negative,pruned_base_adj_false_negative,cluster_adj_false_negative
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,17.423611,17.423611,16.737500,13.920833,13.920833,10.658333
std,11.310868,11.310868,11.179991,9.794444,9.794444,7.947669
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,7.000000,7.000000,5.000000,5.000000,3.750000
50%,18.000000,18.000000,17.000000,13.000000,13.000000,10.000000
75%,27.000000,27.000000,26.000000,23.000000,23.000000,17.000000
max,41.000000,41.000000,41.000000,37.000000,37.000000,32.000000


### Comparing total true and false

In [13]:
data['base_total_true'] = data['base_adj_true_positive'] + data['base_adj_true_negative']
data['base_total_false'] = data['base_adj_false_positive'] + data['base_adj_false_negative']
data['pruned_base_total_true'] = data['pruned_base_adj_true_positive'] + data['pruned_base_adj_true_negative']
data['pruned_base_total_false'] = data['pruned_base_adj_false_positive'] + data['pruned_base_adj_false_negative']
data['cluster_total_true'] = data['cluster_adj_true_positive'] + data['cluster_adj_true_negative']
data['cluster_total_false'] = data['cluster_adj_false_positive'] + data['cluster_adj_false_negative']
total_true_false_metrics = ['base_total_true', 'pruned_base_total_true', 'cluster_total_true', \
                    'base_total_false', 'pruned_base_total_false', 'cluster_total_false']
data[total_true_false_metrics].describe()

,base_total_true,pruned_base_total_true,cluster_total_true,base_total_false,pruned_base_total_false,cluster_total_false
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,29.837500,29.837500,32.413889,15.162500,15.162500,12.586111
std,9.536839,9.536839,7.598282,9.536839,9.536839,7.598282
min,8.000000,8.000000,13.000000,0.000000,0.000000,0.000000
25%,21.000000,21.000000,26.000000,6.000000,6.000000,6.000000
50%,31.000000,31.000000,33.000000,14.000000,14.000000,12.000000
75%,39.000000,39.000000,39.000000,24.000000,24.000000,19.000000
max,45.000000,45.000000,45.000000,37.000000,37.000000,32.000000


### Grouping by clusters

In [14]:
data.groupby('n_clusters').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
n_clusters,,,,,,,,,
2.0,0.904223,0.904223,0.888789,0.554902,0.554902,0.617727,0.658700,0.658700,0.704593
3.0,0.909659,0.909659,0.883369,0.556856,0.556856,0.655634,0.661521,0.661521,0.729515
4.0,0.915778,0.915778,0.887349,0.548819,0.548819,0.672270,0.658358,0.658358,0.746043
5.0,0.923331,0.923331,0.903657,0.560084,0.560084,0.701003,0.671428,0.671428,0.772257


In [15]:
data.groupby('n_clusters').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
n_clusters,,,
2.0,18.238889,18.238889,15.777778
3.0,18.333333,18.333333,14.322222
4.0,18.622222,18.622222,13.683333
5.0,17.861111,17.861111,11.883333


### Grouping by alpha

In [16]:
data.groupby('alpha').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
alpha,,,,,,,,,
0.01,0.948728,0.948728,0.942104,0.495971,0.495971,0.604015,0.627841,0.627841,0.721038
0.05,0.936026,0.936026,0.923896,0.528446,0.528446,0.636658,0.651414,0.651414,0.736802
0.10,0.934417,0.934417,0.914715,0.549885,0.549885,0.659844,0.668707,0.668707,0.750928
0.25,0.901721,0.901721,0.871188,0.580214,0.580214,0.683626,0.678450,0.678450,0.744005
0.50,0.845347,0.845347,0.802053,0.621310,0.621310,0.724150,0.686096,0.686096,0.737737


In [17]:
data.groupby('alpha').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
alpha,,,
0.01,18.590278,18.590278,14.159722
0.05,18.104167,18.104167,13.604167
0.10,18.097222,18.097222,13.673611
0.25,17.895833,17.895833,13.638889
0.50,18.631944,18.631944,14.506944


### Grouping by distribution type


In [18]:
data.groupby('distribution_type').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
distribution_type,,,,,,,,,
exp,0.914597,0.914597,0.890225,0.554438,0.554438,0.659878,0.663444,0.663444,0.736271
gauss,0.908928,0.908928,0.887066,0.548577,0.548577,0.659078,0.656581,0.656581,0.736129
gumbel,0.916218,0.916218,0.895083,0.562481,0.562481,0.666020,0.667481,0.667481,0.741906


In [19]:
data.groupby('distribution_type').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
distribution_type,,,
exp,18.125000,18.125000,13.954167
gauss,18.729167,18.729167,14.137500
gumbel,17.937500,17.937500,13.658333


### Grouping by DAG method

In [20]:
data.groupby('dag_method').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
dag_method,,,,,,,,,
erdos_renyi,0.919161,0.919161,0.886709,0.598416,0.598416,0.690485,0.691420,0.691420,0.750722
hierarchical,0.935343,0.935343,0.928999,0.387726,0.387726,0.529399,0.544878,0.544878,0.669588
scale_free,0.885239,0.885239,0.856665,0.679354,0.679354,0.765091,0.751207,0.751207,0.793996


In [21]:
data.groupby('dag_method').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
dag_method,,,
erdos_renyi,17.762500,17.762500,13.379167
hierarchical,25.804167,25.804167,19.987500
scale_free,11.225000,11.225000,8.383333


### Grouping by #clusters

In [22]:
data.groupby('n_clusters').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
n_clusters,,,,,,,,,
2.0,0.904223,0.904223,0.888789,0.554902,0.554902,0.617727,0.658700,0.658700,0.704593
3.0,0.909659,0.909659,0.883369,0.556856,0.556856,0.655634,0.661521,0.661521,0.729515
4.0,0.915778,0.915778,0.887349,0.548819,0.548819,0.672270,0.658358,0.658358,0.746043
5.0,0.923331,0.923331,0.903657,0.560084,0.560084,0.701003,0.671428,0.671428,0.772257


In [23]:
data.groupby('n_clusters').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
n_clusters,,,
2.0,18.238889,18.238889,15.777778
3.0,18.333333,18.333333,14.322222
4.0,18.622222,18.622222,13.683333
5.0,17.861111,17.861111,11.883333


### Grouping by #edges

In [24]:
data.groupby('n_edges').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
n_edges,,,,,,,,,
10.0,0.890428,0.890428,0.856513,0.726147,0.726147,0.789022,0.761183,0.761183,0.791092
20.0,0.911490,0.911490,0.885739,0.597950,0.597950,0.700763,0.703544,0.703544,0.768241
30.0,0.904291,0.904291,0.885407,0.490978,0.490978,0.612445,0.626857,0.626857,0.714589
50.0,0.946782,0.946782,0.935506,0.405586,0.405586,0.544403,0.558422,0.558422,0.678487


In [25]:
data.groupby('n_edges').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
n_edges,,,
10.0,10.950000,10.950000,9.016667
20.0,15.322222,15.322222,11.405556
30.0,19.961111,19.961111,14.900000
50.0,26.822222,26.822222,20.344444
